In [1]:
# Imports
import pandas as pd
import numpy as np
import docx
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df = pd.read_csv('PSA Trial Logs.csv')
df.head()

,DATE,LOCATION,EVENT,LEVEL,HANDLER,DOG,BREED,OB,PR,TOTAL,STATUS
0,09.17.23,"HOBART, INDIANA",TRIAL,PSA3,ALYSSA BYARS,ANUBIS,DOBERMAN,15.0,0.0,15.0,NO PASS (BITE)
1,09.17.23,"HOBART, INDIANA",TRIAL,PSA3,KELLIE VANNUIL,MARVEL,DUTCH SHEPHERD,9.0,0.0,9.0,NO PASS (BITE)
2,09.17.23,"HOBART, INDIANA",TRIAL,PDC,SCOTT SHARFMAN,ESME,GERMAN SHEPHERD,57.5,74.0,131.5,PASS
3,09.17.23,"HOBART, INDIANA",TRIAL,PDC,ANNA SAUDER,BLAST,BELGIAN MALINOIS,56.0,81.5,137.5,PASS
4,09.17.23,"HOBART, INDIANA",TRIAL,PDC,LIDIJA BELL,JAX,GERMAN SHEPHERD,55.5,72.5,128.0,PASS


In [3]:
"""Clean up the data a bit"""
def data_cleanse(df):
    # Drop any NaNs, as we don't really have insight as to why
    df = df.dropna(subset='STATUS')
    # Making this column a string, because I got an error stating that it wasn't
    df['STATUS'] = df['STATUS'].astype(str)
    # Consolidating the NO PASS BITE String to minimze repeat values
    df['STATUS'] = df['STATUS'].mask(df['STATUS'].str.contains('BITE') == True, 'NO PASS (BITE)')
    # Consolidating the NO PASS NO OUT String to minimze repeat values
    df['STATUS'] = df['STATUS'].mask(df['STATUS'].str.contains('OUT') == True, 'NO PASS (NO OUT)')
    # Consolidating the NO PASS NO OUT String to minimze repeat values
    df['LEVEL'] = df['LEVEL'].mask(df['LEVEL'].str.contains('SD') == True, 'PDC_SD')
    # Get rid of status ACTIVE
    df = df.loc[df['STATUS'] != 'ACTIVE']
    # Get rid of status REGISTERED
    df = df.loc[df['STATUS'] != 'REGISTERED']
    # Return the DF
    return df

In [4]:
"""Cleaning/grouping/resampling function to create generically useable data"""
def groupby_counts(base_df, column_name):
  # Create a new frame to group the unique values by size
  df = base_df.groupby([column_name]).size()
  # Reassign the new frame to create the new column
  df = base_df.join(df.to_frame(), on=column_name)
  # Drop all duplicate entries, leaving only the unique entries
  df = df.drop_duplicates(subset=[column_name])
  # New df to not have to list all columns to drop
  finished_df = df[[column_name, 0]]
  # Rename the column
  finished_df = finished_df.rename(columns={0 : f'{column_name}_COUNTS'})
  # Return newly formatted df
  return finished_df

In [5]:
"""Lets go with details about the different levels of PSA, to include breed, scores, and status"""
def psa_level_details(base_df, psa_level):
    # Cleanse the DF for consistent results
    df = data_cleanse(base_df)
    # Separate the frame to only fit the level
    psa_level_df = df.loc[df['LEVEL'] == psa_level]
    # Now, lets see some numbers on the status
    psa_level_counts = groupby_counts(psa_level_df, 'STATUS')
    # Change column name of the counts to include the psa level
    psa_level_counts = psa_level_counts.rename(columns={'STATUS_COUNTS':f'{psa_level}_STATUS_COUNTS'})
    # Change to int rather than float (preference)
    psa_level_counts[f'{psa_level}_STATUS_COUNTS'] = psa_level_counts[f'{psa_level}_STATUS_COUNTS'].astype(int)
    # Sort the items by the counts
    psa_level_counts = psa_level_counts.sort_values(by=f'{psa_level}_STATUS_COUNTS', ascending=False)
    # Return the deliverables
    return psa_level_counts

In [6]:
"""Let's also get some info on most common breeds"""
def most_popular_breeds():
    return

In [15]:
"""Let's try and get used to using docx"""
def build_word_doc(df):
    return

# Make a test dataframe
test_df = psa_level_details(df, 'PSA3')

# Init document
doc = docx.Document()

# Initialise Table
t = doc.add_table(rows=test_df.shape[0] + 1, cols=test_df.shape[1])

# Add borders
t.style = 'TableGrid'
# Add the column headings
for j in range(test_df.shape[1]):
    t.cell(0, j).text = test_df.columns[j]

# Add the body of the data frame to the table
for i in range(test_df.shape[0]):
    for j in range(test_df.shape[1]):
        cell = test_df.iat[i, j]
        t.cell(i + 1, j).text = str(cell)

doc.save('table 1.docx')

test_df

c:\Users\My PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\docx\styles\styles.py:139: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


,STATUS,PSA3_STATUS_COUNTS
18,NO PASS,234
0,NO PASS (BITE),146
252,PASS,56
135,PULL,29
1858,DQ,4
74,NO PASS (NO OUT),3
485,NO SHOW,2
